<a href="https://colab.research.google.com/github/AditiLORenzo/Aditiprojects/blob/main/EDA_on_Text_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import math

Import Data

In [ ]:
df=pd.read_csv('/content/news_syn (3).csv')
df.shape

In [ ]:
df

In [ ]:
df.describe(include='object')

Check wheather there is any null values

In [ ]:
df.isnull().sum().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df['Article category'].shape[0]

In [ ]:
#df=df.groupby('Article category ').filter(lambda x:len(x)>500).reset_index(drop=True)
#print("number of article>=",len(df['Article category'].unique()))
df=df.groupby('Article category').filter(lambda x:len(x)<500).reset_index(drop=True)

In [ ]:
print("number of article>=",len(df['Article category'].unique()))

In [ ]:
df['Article category'].unique()

In [ ]:
# converting categorical data to integers
#df['short_description']=df['short_description'].astype(int)

In [ ]:
len(df['News Headline'].unique())

In [ ]:
df

Cleaning Text data

In [ ]:
df['News Headline']=df['News Headline'].apply(lambda x:x.split(',,,')[0])

In [ ]:
df['News Headline']

In [ ]:
len(df['short_description'].unique())

In [ ]:
for index,text in enumerate(df['News Headline'][18:23]):
    print('News headlines%d:\n'%(index+1),text)

In [ ]:
for index,text in enumerate(df['short_description'][35:40]):
    print('Short description%d:\n'%(index+1),text)

In [ ]:
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

In [ ]:
#regular expression for finding the contraction
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

df['News Headline']=df['News Headline'].apply(lambda x:expand_contractions(x))  
df['short_description']=df['short_description'].apply(lambda x:expand_contractions(x))

Lower case 

In [ ]:
df['News Headline']=df['News Headline'].apply(lambda x: x.lower())
df['short_description']=df['short_description'].apply(lambda x: x.lower())

removing digits

In [ ]:
df['News Headline']=df['News Headline'].apply(lambda x: re.sub('\w*\d\w*','', x))
df['short_description']=df['short_description'].apply(lambda x: re.sub('\w*\d\w*','',x))

Remove puntuation

In [ ]:
df['News Headline']=df['News Headline'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
df['short_description']=df['short_description'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

performing EDA

In [ ]:
import spacy

# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# Lemmatization with stopwords removal
df['lemmatized1']=df['News Headline'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
df['lemmatized2']=df['short_description'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

In [ ]:
df_grouped=df[['News Headline','short_description','lemmatized1','lemmatized2']].groupby(by='News Headline').agg(lambda x:' '.join(x))
df_grouped.head()

Creating document term matrix

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(analyzer='word')
data=cv.fit_transform(df_grouped['lemmatized1'])
df_dtm = pd.DataFrame(data.toarray(), columns=cv.get_feature_names())
df_dtm.index=df_grouped.index
df_dtm.head(3)

In [ ]:
cv=CountVectorizer(analyzer='word')
data=cv.fit_transform(df_grouped['lemmatized2'])
df_dtm = pd.DataFrame(data.toarray(), columns=cv.get_feature_names())
df_dtm.index=df_grouped.index
df_dtm.head(3)

In [ ]:
from textwrap import wrap



In [ ]:
#function for generating word cloud
from wordcloud import WordCloud 
def generate_wordcloud(data,title):
    wc = WordCloud(width=400, height=330, max_words=150,colormap="Dark2").generate_from_frequencies(data)
    plt.figure(figsize=(10,8))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    plt.title('\n'.join(wrap(title,60)),fontsize=13)
    plt.show()

In [ ]:
df_dtm=df_dtm.transpose()

In [ ]:
for index,product in enumerate(df_dtm.columns):
    generate_wordcloud(df_dtm[product].sort_values(ascending=False),product)

In [ ]:
from textblob import TextBlob
df['polarity1']=df['lemmatized1'].apply(lambda x:TextBlob(x).sentiment.polarity)
df['polarity2']=df['lemmatized2'].apply(lambda x:TextBlob(x).sentiment.polarity)

In [ ]:
#print("10 Random News headline with Highest Polarity:")
for index,review in enumerate(df.iloc[df['polarity1'].sort_values(ascending=False)[:10].index]['News Headline']):
    print('News Headline {}:\n'.format(index+1),review)

In [ ]:
print("5 Random short_description with Highest Polarity:")
for index,review in enumerate(df.iloc[df['polarity2'].sort_values(ascending=False)[:5].index]['short_description']):
  print('short description {}:\n'.format(index+1),review)

In [ ]:
print("5 Random short description with Lowest Polarity:")
for index,review in enumerate(df.iloc[df['polarity1'].sort_values(ascending=True)[:5].index]['short_description']):
  print('Review {}:\n'.format(index+1),review)

In [ ]:
print("5 Random news headline with Lowest Polarity:")
for index,review in enumerate(df.iloc[df['polarity2'].sort_values(ascending=True)[:5].index]['News Headline']):
  print('Review {}:\n'.format(index+1),review)

lets plot polarity result

In [ ]:
product_polarity_sorted=pd.DataFrame(df.groupby('short_description')['polarity2'].mean().sort_values(ascending=True))

plt.figure(figsize=(16,8))
plt.xlabel('Polarity')
plt.ylabel('description')
plt.title('Polarity of News short description')
polarity_graph=plt.barh(np.arange(len(product_polarity_sorted.index)),product_polarity_sorted['polarity2'],color='purple',)

# Writing product names on bar
for bar,product in zip(polarity_graph,product_polarity_sorted.index):
  plt.text(0.005,bar.get_y()+bar.get_width(),'{}'.format(product),va='center',fontsize=11,color='white')

# Writing polarity values on graph
for bar,polarity in zip(polarity_graph,product_polarity_sorted['polarity2']):
  plt.text(bar.get_width()+0.001,bar.get_y()+bar.get_width(),'%.3f'%polarity,va='center',fontsize=11,color='black')
  
plt.yticks([])
plt.show()

In [ ]:
product_polarity_sorted=pd.DataFrame(df.groupby('News Headline')['polarity1'].mean().sort_values(ascending=True))

plt.figure(figsize=(16,8))
plt.xlabel('Polarity')
plt.ylabel('description')
plt.title('Polarity of News short description')
polarity_graph=plt.barh(np.arange(len(product_polarity_sorted.index)),product_polarity_sorted['polarity1'],color='purple',)

# Writing product names on bar
for bar,product in zip(polarity_graph,product_polarity_sorted.index):
  plt.text(0.005,bar.get_y()+bar.get_width(),'{}'.format(product),va='center',fontsize=11,color='white')

# Writing polarity values on graph
for bar,polarity in zip(polarity_graph,product_polarity_sorted['polarity1']):
  plt.text(bar.get_width()+0.001,bar.get_y()+bar.get_width(),'%.3f'%polarity,va='center',fontsize=11,color='black')
  
plt.yticks([])
plt.show()